# Frigate Models (Colab)
Downloads and converts SSDLite MobileNet v2 (OpenVINO IR), and downloads YOLOv9 + YOLO-NAS weights.

In [3]:
# Base system deps for native wheels: use py 3.11 runtime
!apt-get -qq update
!apt-get -qq install -y wget python3.11 python3-dev python3-distutils gcc pkg-config libhdf5-dev

!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 2
!update-alternatives --set python3 /usr/bin/python3.11

!wget -q https://bootstrap.pypa.io/get-pip.py -O get-pip.py
!sed -i 's/args.append("setuptools")/args.append("setuptools==77.0.3")/' get-pip.py

!python3.11 get-pip.py "pip"
!python3.11 -m pip install "numpy<2.0" tensorflow "openvino-dev>=2024.6.0"

import sys
print(sys.version)
print(sys.executable)

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
  Using cached pip-25.3-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.3-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 25.3
    Uninstalling pip-25.3:
      Successfully uninstalled pip-25.3
3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
/usr/bin/python3


In [4]:
!wget http://download.tensorflow.org/models/object_detection/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz
!tar -xvf ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz

--2026-01-24 23:24:06--  http://download.tensorflow.org/models/object_detection/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.31.207, 142.251.111.207, 142.251.16.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.31.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51025348 (49M) [application/x-tar]
Saving to: ‘ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz.1’

ssdlite_mobilenet_v 100%[===================>]  48.66M   222MB/s    in 0.2s    

2026-01-24 23:24:07 (222 MB/s) - ‘ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz.1’ saved [51025348/51025348]

ssdlite_mobilenet_v2_coco_2018_05_09/checkpoint
ssdlite_mobilenet_v2_coco_2018_05_09/model.ckpt.data-00000-of-00001
ssdlite_mobilenet_v2_coco_2018_05_09/model.ckpt.meta
ssdlite_mobilenet_v2_coco_2018_05_09/model.ckpt.index
ssdlite_mobilenet_v2_coco_2018_05_09/saved_model/saved_model.pb
ssdlite_mobilenet_v2_coco_2018_05_09/

In [5]:

import openvino as ov
from openvino.tools import mo

ov_model = mo.convert_model(
    "ssdlite_mobilenet_v2_coco_2018_05_09/frozen_inference_graph.pb",
    compress_to_fp16=True,
    transformations_config="/usr/local/lib/python3.11/dist-packages/openvino/tools/mo/front/tf/ssd_v2_support.json",
    tensorflow_object_detection_api_pipeline_config="ssdlite_mobilenet_v2_coco_2018_05_09/pipeline.config",
    reverse_input_channels=True,
)
ov.save_model(ov_model, "/models/ssdlite_mobilenet_v2.xml")

[ INFO ] MO command line tool is considered as the legacy conversion API as of OpenVINO 2023.2 release.
In 2025.0 MO command line tool and openvino.tools.mo.convert_model() will be removed. Please use OpenVINO Model Converter (OVC) or openvino.convert_model(). OVC represents a lightweight alternative of MO and provides simplified model conversion API. 
Find more information about transition from MO to OVC at https://docs.openvino.ai/2023.2/openvino_docs_OV_Converter_UG_prepare_model_convert_model_MO_OVC_transition.html


ERROR:root:The Preprocessor block has been removed. Only nodes performing mean value subtraction and scaling (if applicable) are kept.


In [ ]:
%cd /content
!test -d /content/yolov9 || git clone -q https://github.com/WongKinYiu/yolov9.git
%cd /content/yolov9
!pip -q install -r requirements.txt
!wget -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt
!wget -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e.pt
!python export.py --weights yolov9-c.pt --img 640 --batch 1 --device cpu --include onnx
!python export.py --weights yolov9-e.pt --img 640 --batch 1 --device cpu --include onnx
!ls -lh *.onnx


In [ ]:
%cd /content
# YOLO-NAS -> ONNX (avoid source builds)
!pip -q install --upgrade pip setuptools wheel
!pip -q install --only-binary :all: onnx==1.14.1 pycocotools==2.0.7
!pip -q install super-gradients
from super_gradients.training import models
from super_gradients.common.object_names import Models
model = models.get(Models.YOLO_NAS_S, pretrained_weights="coco")
model.export("yolo_nas_s.onnx", input_size=(640, 640), batch_size=1)
!ls -lh yolo_nas_s.onnx


In [ ]:
%cd /content
!mkdir -p artifacts
!cp /tmp/omz/models/public/ssdlite_mobilenet_v2/FP16/ssdlite_mobilenet_v2.* artifacts/
!cp /content/yolov9/*.onnx artifacts/
!cp /content/yolo_nas_s.onnx artifacts/
!ls -lh artifacts
!zip -r frigate_models.zip artifacts


In [ ]:
from google.colab import files
files.download("frigate_models.zip")
